*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

You should create a function called `predict_message` that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the [SMS Spam Collection dataset](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/). The dataset has already been grouped into train data and test data.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.


In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
train_ds = pd.read_csv(train_file_path, sep='\t', header=None, names=['label', 'message'])
train_ds

In [ ]:
train_ds.label.value_counts()

In [ ]:
train_ds.label = train_ds.label.map({'spam': 1, 'ham': 0})
train_ds

In [ ]:
test_ds = pd.read_csv(test_file_path, sep='\t', header=None, names=['label', 'message'])
test_ds.label = test_ds.label.map({'spam': 1, 'ham': 0})
test_ds

In [ ]:
x_train = train_ds.message.values
y_train = train_ds.label.values
x_test = test_ds.message.values
y_test = test_ds.label.values

In [ ]:
# text preprocessing: padding and vectorization
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer

#create vocabulary from train dataset
t = Tokenizer()
t.fit_on_texts(x_train)

#encode the text 
encode_train = t.texts_to_sequences(x_train)
encode_test = t.texts_to_sequences(x_test)

#pad the text to the same length
length = 10
pad_train = pad_sequences(encode_train, maxlen=length, padding='post')
pad_test = pad_sequences(encode_test, maxlen=length, padding='post')

In [ ]:
from keras import layers
#bulid model
model = tf.keras.Sequential([
  layers.Embedding(len(t.word_index) + 1, 36, input_length=length),
  layers.Dropout(0.5),
  layers.GlobalAveragePooling1D(),
  layers.Dropout(0.5),
  layers.Dense(1, activation='sigmoid')])

model.summary()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
#train 
model.fit(x=pad_train,
         y=y_train,
         epochs=30,
         validation_data=(pad_test, y_test), verbose=1
         )

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  encode_pred_text = t.texts_to_sequences([pred_text])
  pad_pred_text = pad_sequences(encode_pred_text, maxlen=length, padding='post')
  p = model.predict(pad_pred_text.reshape(1, length)).astype('float32')
  pred_c = (model.predict(pad_pred_text.reshape(1, length))>.2).astype('int32')
  if pred_c == 0:
    prediction = [p, 'ham']
  else:
    prediction = [p, 'spam']

  return (prediction)

pred_text = "sale today! to stop texts call 98912460324"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
